In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Import dependencies

In [2]:
import os
import random
import numpy as np
import pandas as pd
from keras import optimizers
from keras import applications
from keras.layers import Input
from keras import initializers
from keras.optimizers import Adam
from keras.models import load_model
from keras.models import Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D, MaxPooling2D
from keras.layers.core import Reshape, Dense, Dropout, Flatten, Activation

Using TensorFlow backend.


The dataset consists of a total of 300 images. 50 images of 6 classes each. We use 80% of these images for training our model and remaining 20% for testing the model. For splitting the dataset, we generate 60 random numbers in the range of 0 to 299. These numbers are used as indexs for retreiving images from the dataset which is in the form of a numpy array. Since the images are not stored sequentially in the dataset, this splits the dataset in a way that samples of all the classes are equally selected.





In [3]:
os.chdir('/content/drive/My Drive/Colab Notebooks/Club logo classification/')
path = os.getcwd()

data = np.load('data.npy')
labels = np.load('target_labels.npy')

def test_train_split(data):
    test_samples = list(random.sample(range(0, 300), 60))
    train_samples = []
    for i in range(0, 300):
        if i not in test_samples:
            train_samples.append(i)
    temp = []
    for i in train_samples:
        temp.append(data[i])
    train_x = np.asarray(temp)
    temp = []
    for i in train_samples:
        temp.append(labels[i])
    train_y = np.asarray(temp)
    temp = []
    for i in test_samples:
        temp.append(data[i])
    test_x = np.asarray(temp)
    temp = []
    for i in test_samples:
        temp.append(labels[i])
    test_y = np.asarray(temp)

    print(train_x.shape)
    print(train_y.shape)
    print(test_x.shape)
    print(test_y.shape)
    return train_x, train_y, test_x, test_y

train_x, train_y, test_x, test_y = test_train_split(data)

(240, 224, 224, 3)
(240,)
(60, 224, 224, 3)
(60,)


By using Transfer Learning, we can build our model on top of an exsting pre-trained model. This method is effective when we want to extract higher level features from a relatively small dataset. We use the MobileNet_v2 model pre-trained on the Imagenet dataset. We remove the fully connected layer of the model which is trained to classify 1000 classes and replace with our fully conected layer to classify  6 classes.

In [9]:
base_model = applications.mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(6, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

9412608/9406464 [==============================] - 1s 0us/step


Train the model for 25 epochs with a batch_size of 5 and 20% validation split

In [10]:
model.fit(train_x, train_y, batch_size = 5, validation_split = 0.2, epochs = 25)

Train on 192 samples, validate on 48 samples
Epoch 1/25
192/192 [==============================] - 17s 89ms/step - loss: 1.2548 - acc: 0.5469 - val_loss: 0.5846 - val_acc: 0.8333
Epoch 2/25
192/192 [==============================] - 3s 15ms/step - loss: 0.3749 - acc: 0.9115 - val_loss: 0.4470 - val_acc: 0.8542
Epoch 3/25
192/192 [==============================] - 3s 15ms/step - loss: 0.1972 - acc: 0.9323 - val_loss: 0.4928 - val_acc: 0.8542
Epoch 4/25
192/192 [==============================] - 3s 15ms/step - loss: 0.0510 - acc: 0.9844 - val_loss: 0.2953 - val_acc: 0.8958
Epoch 5/25
192/192 [==============================] - 3s 15ms/step - loss: 0.0710 - acc: 0.9792 - val_loss: 0.3510 - val_acc: 0.8750
Epoch 6/25
192/192 [==============================] - 3s 15ms/step - loss: 0.0861 - acc: 0.9635 - val_loss: 0.2571 - val_acc: 0.9167
Epoch 7/25
192/192 [==============================] - 3s 15ms/step - loss: 0.0304 - acc: 0.9896 - val_loss: 0.2989 - val_acc: 0.8958
Epoch 8/25
192/192 [===

Test the accuracy of the model

In [12]:
scores = model.evaluate(test_x, test_y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

60/60 [==============================] - 0s 2ms/step
acc: 91.67%


Save the model in .h5 format for future use

In [0]:
model.save('model.h5')